Thakaa center challenge

In [1]:
path1 = "Workable JD/workable.csv" #path to read JD from file Workable JD 
data =[]

In [2]:
with open(path1) as f:
    for line in f:
        linesplit =line.split(",")
        
        data.append(linesplit)

In [4]:
print (data[1])

['1', 'Accounting', 'Cost analyst', ' Cost analysts study corporate expenses and create reports for management. One common industry for cost analysts is manufacturing where they are needed to audit production costs. Similar job titles are cost specialist and cost accountant. ', ' The primary duty of a cost analyst is to gather financial data (e.g. inventory purchases  labor costs) and analyze it to uncover inefficiencies. They may estimate production costs and track the changes in regular costs (e.g. due to new products and new processes).', 'Collecting and analyzing financial data Estimating costs and comparing budgets with actual expenses Monitoring costs after significant changes in products or processes', 'We are looking for a cost analyst to help us audit our expenses and find ways to make our operations more cost-efficient. You’ll be the go-to person for cost analysis and you’ll get to prepare reports to help management make better decisions. To do this job well  we’d like you to

In [5]:
path2 = "Open source global resumes"
import sys,os
resumes=[]  # resumes[[ID][Source][Path][file_name][Extension][Resume_Object]]
Directories =[]
path= './' + path2


In [7]:
# resumes[[ID][Source][Path][file_name][Extension][Resume_Object]]
def fileExtension(file_name):
    if 'pdf' in file_name:
        return 'pdf'
    elif 'docx' in file_name:
        return 'docx'
    elif 'doc' in file_name:
        return 'doc'
    else:
        return 'unknown'
ID = 0
for root, subFolders, files in os.walk(path ):#+'/Random'
    print  (root)
    print ('-------------------')
    #print files
    for file_name in files:
        print ('X = ' + file_name)
        ID = ID + 1
        Source = 'Open source global resumes'
        resume = [ID,Source,root,file_name,fileExtension(file_name)]
        resumes.append(resume)

./Open source global resumes
-------------------
X = Hedge Fund Accountant JD.docx
X = 4Dickson.pdf
X = 7Giraffe.pdf
X = 7Giraffe.doc
X = 4Dickson.doc
X = 6Faye.pdf
X = 9Ignatius.pdf
X = .DS_Store
X = 2Ben.pdf
X = a2.pdf
X = 1Amy.docx
X = 5Edwardo.doc
X = 3Carrie.pdf
X = 2Ben.docx
X = 9Ignatius.docx
X = antiword.exe
X = 5Edwardo.pdf
X = 3Carrie.docx
X = 6Faye.docx
X = a1.pdf
X = Major_Project_Report.pdf
X = 1Amy.pdf
X = 8Holly.pdf
X = 10Jonas.doc
X = 10Jonas.pdf
X = 8Holly.docx
./Open source global resumes/Random
-------------------
X = State Street_Joseph Falvella_PE Sales.pdf
X = Sansar_Adrian Bek_Controller.doc
X = AEW Capital_Desmond Tay_Portfolio Accountant_RE.docx
X = CITCO_Jeremy Lu_Supervisor_Hedge.pdf
X = CITCO_Ong Tiong Guan_Fund Accountant_Hedge.docx
X = Goldman_Thariq Razeen_Trade Mgmt Associate.doc
X = SMBC_Tra Zboinski_VP Corp Solutions.docx
X = Fusang_Val Yoh_Ops SM.doc
X = HSBC Trustee_Sylvia Lau_Trust Accountant.pdf
X = CDL_Jannie Do_Executive.doc
X = Blackrock_June Ho

./Open source global resumes/Operations
-------------------
./Open source global resumes/Operations/Neo Criterion HK - Ops Mgr
-------------------
X = JPM_Carman Yau_Fund Analyst.doc
X = Religare_Kiki Wang_Middle Office Analyst.doc
./Open source global resumes/Operations/Voyager HK - RO
-------------------
X = VMS_Gary Chan_MD.doc
X = Ark Pacific_Lam Sai Hou_Sr Ops Mgr.doc
./Open source global resumes/Operations/Principal Global SG - Ops
-------------------
X = Eastspring_Alex Choo_Trade Support.doc
X = SS&C_Jonathan Arambam_Ops Mgr_Hedge.doc
X = Harvest FM_Shan Pengyi_Fund Acc AVP.docx
X = UOB_Jean Tan_Margin Credit.docx
X = Natixis_Ero Sam_Ops Officer.doc
X = Amundi_Jane Low_Sr Exec.pdf
X = Goldman Sachs_Sally Kor_Ops Analyst.pdf
X = Kiara_Thandar MyoAung_Sr Ops_Hedge.doc
./Open source global resumes/Operations/Kayak SG - GM
-------------------
X = Syniverse Technologies_Karen Sim_Exec Dir.docx
./Open source global resumes/Operations/CITCO SG - Pricing AVP
-------------------
X = SCB

In [9]:
print (len (resumes))
print (resumes[0])

2170
[1, 'Open source global resumes', './Open source global resumes', 'Hedge Fund Accountant JD.docx', 'docx']


In [10]:
from docx import Document
from docx.shared import Inches
import PyPDF2 as PY